In [45]:
import pandas as pd

data = pd.read_csv(
    "/Users/cordiez/Documents/CompaniesHouse/OutputFiles/tc_scrapped_final.csv",
    index_col=0,
)
tech5g = data[~data["content"].isna()].reset_index(drop=True)

In [46]:
from tqdm import tqdm_notebook

tqdm_notebook().pandas()

In [47]:
import re

# Gensim
import gensim
import gensim.corpora as corpora
import pandas as pd
# spacy for lemmatization
import spacy
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

nlp = spacy.load("en_core_web_sm")

# Plotting tools

%matplotlib inline

# Enable logging for gensim - optional
import logging

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.ERROR
)

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

stop_words = stopwords.words("english")
# stop_words.extend(['milton','keynes','world','work','uk','support','service','search','contact','shop','usage','business', 'manage','provide','website','help','product','solution','company','design','get','team','experience','customer','need','client','develop','quick','sale','day','ltd','project','work','office','market','can','will','offer','find','new','year','look','call','see','one','service','news',"news","events","press","contact","search"])

In [49]:
tech5g.head()

,title,url,author,content,summary,url_exact
0,Intel announces two new 11th-gen chips and a 5...,https://search.techcrunch.com/click/_ylt=A2KLf...,Catherine Shu,Intel kicked off this year’s virtual Computex ...,Intel kicked off this year’s virtual Computex ...,https://techcrunch.com/2021/05/30/intel-announ...
1,China expresses concern over its absence in In...,https://search.techcrunch.com/click/_ylt=A2KLf...,Manish Singh,China expressed concern on Wednesday over Indi...,China expressed concern on Wednesday over Indi...,https://techcrunch.com/2021/05/05/china-expres...
2,"India grants approval for 5G trials, avoids Ch...",https://search.techcrunch.com/click/_ylt=A2KLf...,Manish Singh,The Indian telecom ministry on Tuesday said it...,The Indian telecom ministry on Tuesday said it...,https://techcrunch.com/2021/05/04/india-grants...
3,Verizon and Honda want to use 5G and edge comp...,https://search.techcrunch.com/click/_ylt=A2KLf...,Rebecca Bellan,Honda and Verizon are researching how 5G and m...,Honda and Verizon are researching how 5G and m...,https://techcrunch.com/2021/04/08/verizon-and-...
4,"Google denies Pixel 5a 5G cancelation, confirm...",https://search.techcrunch.com/click/_ylt=A2KLf...,Brian Heater,Sometimes you’ve just got to confirm an unanno...,Sometimes you’ve just got to confirm an unanno...,https://techcrunch.com/2021/04/09/google-denie...


In [50]:
data = tech5g.content.tolist()

data = [re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", sent) for sent in data]

# Remove Emails
data = [re.sub("\S*@\S*\s?", "", sent) for sent in data]
# Remove new line characters
data = [re.sub("\s+", " ", sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("'", "", sent) for sent in data]
data = [sent.replace("/", " ") for sent in data]


def sent_to_words(sentences):
    for sentence in sentences:
        yield (
            gensim.utils.simple_preprocess(str(sentence), deacc=True)
        )  # deacc=True removes punctuations


data_words = list(sent_to_words(data))

In [51]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(
    data_words, min_count=5, threshold=100
)  # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example

In [53]:
def remove_stopwords(texts):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags]
        )
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(
    data_words_bigrams, allowed_postags=["NOUN", "ADJ", "ADV"]
)

# print(data_lemmatized[:1])

In [54]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

In [41]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    passes=20,
    alpha="auto",
    per_word_topics=True,
)

In [44]:
result = lda_model.print_topics(num_topics=5, num_words=7)
for elt in result:
    print(elt)

(0, '0.018*"network" + 0.013*"government" + 0.010*"technology" + 0.010*"huawei" + 0.009*"also" + 0.009*"chinese" + 0.006*"company"')
(1, '0.009*"startup" + 0.008*"technology" + 0.008*"also" + 0.007*"company" + 0.007*"datum" + 0.007*"tech" + 0.006*"investment"')
(2, '0.010*"smartphone" + 0.009*"also" + 0.009*"mobile" + 0.009*"device" + 0.007*"network" + 0.006*"time" + 0.006*"last"')
(3, '0.010*"app" + 0.010*"mobile" + 0.008*"apple" + 0.008*"game" + 0.007*"phone" + 0.006*"also" + 0.006*"user"')
(4, '0.016*"phone" + 0.014*"device" + 0.010*"also" + 0.009*"camera" + 0.008*"apple" + 0.007*"well" + 0.007*"time"')


In [90]:
x_ = []
y_ = []

for k in range(1, 45):
    print(k)
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=k,
        passes=10,
        alpha="auto",
        per_word_topics=True,
    )

    cm = coherence_model_lda = CoherenceModel(
        model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence="c_v"
    )

    coherence = cm.get_coherence()
    print(coherence)
    x_.append(k)
    y_.append(coherence)

1
0.30803615018830544
2
0.2800498744601332
3
0.34267629817372414
4
0.3526942989930379
5
0.3473076256594023
6
0.33154005460516656
7
0.34325223330081134
8
0.3470269070630938
9
0.3134360164953185
10
0.3342557386355566
11
0.34341065759259093
12
0.35839738619370926
13
0.3340864219487623
14
0.3417778529998977
15
0.33395183518190463
16
0.3109579775364316
17
0.3429212816656935
18
0.36463976604750065
19
0.3516073290200528
20
0.3485479024360676
21
0.30833027426023185
22
0.3366567541493184
23
0.3333328484068206
24
0.32958301277608565
25
0.3333813008338168
26
0.33059003383545343
27
0.36202480717095137
28
0.3420304905589074
29
0.3380537363696043
30
0.3215847834588549
31
0.3191973210890688
32
0.3251972336207991
33
0.3221909869817509
34
0.31205117260661297
35
0.3372524141933204
36
0.33670863657618416
37
0.318901416926615
38
0.33601608163749097
39
0.30866966566261406
40
0.30921669869895696
41
0.34512633668317716
42
0.3402420115409671
43
0.3450099029497618
44
0.32787071587868266


In [91]:
import plotly.express as px

px.line(x=x_, y=y_, labels={"x": "Number of topics", "y": "Topic Coherence Score"})

In [92]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=18,
    passes=50,
    alpha="auto",
    per_word_topics=True,
)

In [94]:
result = lda_model.print_topics(num_topics=18, num_words=10)
result

[(0,
  '0.018*"company" + 0.014*"technology" + 0.014*"network" + 0.011*"new" + 0.009*"also" + 0.009*"service" + 0.008*"year" + 0.007*"business" + 0.007*"chip" + 0.006*"market"'),
 (1,
  '0.023*"phone" + 0.016*"device" + 0.012*"company" + 0.011*"camera" + 0.010*"new" + 0.010*"also" + 0.010*"year" + 0.008*"well" + 0.008*"big" + 0.008*"time"'),
 (2,
  '0.056*"qualcomm" + 0.021*"company" + 0.015*"shareholder" + 0.013*"broadcom" + 0.013*"acquisition" + 0.011*"board" + 0.011*"deal" + 0.010*"chip" + 0.009*"transaction" + 0.009*"chipmaker"'),
 (3,
  '0.028*"year" + 0.022*"huawei" + 0.020*"company" + 0.016*"government" + 0.016*"chinese" + 0.016*"smartphone" + 0.013*"market" + 0.012*"network" + 0.010*"sale" + 0.009*"quarter"'),
 (4,
  '0.011*"project" + 0.009*"open_stack" + 0.008*"company" + 0.008*"kubernete" + 0.007*"presentation" + 0.007*"foundation" + 0.006*"first" + 0.006*"event" + 0.005*"key" + 0.005*"year"'),
 (5,
  '0.026*"apple" + 0.025*"company" + 0.018*"new" + 0.014*"device" + 0.014*"p

In [63]:
cm = coherence_model_lda = CoherenceModel(
    model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence="c_v"
)

coherence = cm.get_coherence()
coherence

0.3557523299800373

In [96]:
counts = dict()
for i in range(18):
    counts[i] = 0

for elt in corpus:
    tops = lda_model.get_document_topics(elt)
    topic_max = 0
    maxi = 0
    for t in tops:
        if t[1] > maxi:
            maxi = t[1]
            topic_max = t[0]

    counts[topic_max] += 1

In [97]:
ch = """[(0,
  '0.018*"company" + 0.014*"technology" + 0.014*"network" + 0.011*"new" + 0.009*"also" + 0.009*"service" + 0.008*"year" + 0.007*"business" + 0.007*"chip" + 0.006*"market"'),
 (1,
  '0.023*"phone" + 0.016*"device" + 0.012*"company" + 0.011*"camera" + 0.010*"new" + 0.010*"also" + 0.010*"year" + 0.008*"well" + 0.008*"big" + 0.008*"time"'),
 (2,
  '0.056*"qualcomm" + 0.021*"company" + 0.015*"shareholder" + 0.013*"broadcom" + 0.013*"acquisition" + 0.011*"board" + 0.011*"deal" + 0.010*"chip" + 0.009*"transaction" + 0.009*"chipmaker"'),
 (3,
  '0.028*"year" + 0.022*"huawei" + 0.020*"company" + 0.016*"government" + 0.016*"chinese" + 0.016*"smartphone" + 0.013*"market" + 0.012*"network" + 0.010*"sale" + 0.009*"quarter"'),
 (4,
  '0.011*"project" + 0.009*"open_stack" + 0.008*"company" + 0.008*"kubernete" + 0.007*"presentation" + 0.007*"foundation" + 0.006*"first" + 0.006*"event" + 0.005*"key" + 0.005*"year"'),
 (5,
  '0.026*"apple" + 0.025*"company" + 0.018*"new" + 0.014*"device" + 0.014*"pod" + 0.014*"phone" + 0.012*"year" + 0.010*"time" + 0.010*"pixel" + 0.009*"event"'),
 (6,
  '0.019*"network" + 0.012*"carrier" + 0.011*"system" + 0.009*"mobile" + 0.009*"government" + 0.007*"security" + 0.007*"card" + 0.007*"payment" + 0.007*"cell" + 0.005*"also"'),
 (7,
  '0.011*"broadband" + 0.010*"rule" + 0.010*"city" + 0.009*"net_neutrality" + 0.009*"service" + 0.007*"wireless" + 0.007*"company" + 0.007*"internet" + 0.006*"pai" + 0.006*"deployment"'),
 (8,
  '0.013*"device" + 0.010*"network" + 0.010*"connectivity" + 0.010*"sim" + 0.009*"starry" + 0.008*"wireless" + 0.007*"razer" + 0.007*"user" + 0.006*"chipset" + 0.006*"first"'),
 (9,
  '0.028*"app" + 0.020*"game" + 0.018*"apple" + 0.014*"video" + 0.013*"user" + 0.012*"mobile" + 0.010*"new" + 0.010*"gaming" + 0.009*"developer" + 0.008*"platform"'),
 (10,
  '0.027*"mobile" + 0.019*"network" + 0.015*"service" + 0.014*"new" + 0.011*"company" + 0.011*"verizon" + 0.010*"carrier" + 0.009*"tv" + 0.008*"also" + 0.008*"consumer"'),
 (11,
  '0.011*"investment" + 0.011*"tech" + 0.009*"startup" + 0.009*"digital" + 0.009*"also" + 0.008*"new" + 0.008*"industry" + 0.007*"market" + 0.007*"company" + 0.007*"year"'),
 (12,
  '0.013*"misinformation" + 0.009*"company" + 0.009*"claim" + 0.009*"conspiracy" + 0.007*"system" + 0.006*"covid" + 0.006*"vaccine" + 0.006*"internet" + 0.006*"also" + 0.006*"virus"'),
 (13,
  '0.013*"datum" + 0.011*"security" + 0.009*"also" + 0.008*"technology" + 0.008*"tech" + 0.006*"chinese" + 0.005*"risk" + 0.005*"standard" + 0.005*"issue" + 0.005*"device"'),
 (14,
  '0.027*"company" + 0.014*"car" + 0.012*"startup" + 0.009*"vehicle" + 0.009*"year" + 0.008*"also" + 0.007*"new" + 0.007*"technology" + 0.006*"system" + 0.006*"platform"'),
 (15,
  '0.019*"new" + 0.017*"also" + 0.015*"android" + 0.012*"device" + 0.011*"cloud" + 0.010*"feature" + 0.008*"enterprise" + 0.008*"developer" + 0.008*"app" + 0.008*"container"'),
 (16,
  '0.017*"company" + 0.010*"year" + 0.008*"new" + 0.007*"also" + 0.007*"investor" + 0.006*"service" + 0.006*"week" + 0.006*"platform" + 0.006*"firm" + 0.006*"startup"'),
 (17,
  '0.021*"accessibility" + 0.019*"vision" + 0.016*"blind" + 0.012*"technology" + 0.011*"sight" + 0.010*"event" + 0.009*"global" + 0.008*"medium" + 0.007*"session" + 0.007*"tech"')]"""

results = pd.DataFrame()
results["Number of documents"] = None
results["Number of documents"] = None

liste = re.findall(r'"(.*?)"', ch)
liste
for i in range(18):
    x = liste[i * 10 : i * 10 + 10]
    w = ", ".join(x)
    results.loc[i, "Terms"] = w
    results.loc[i, "Number of documents"] = counts[i]

results.style.set_properties(subset=["Terms"], **{"width": "550px"})

,Number of documents,Terms
0,85,"company, technology, network, new, also, service, year, business, chip, market"
1,105,"phone, device, company, camera, new, also, year, well, big, time"
2,10,"qualcomm, company, shareholder, broadcom, acquisition, board, deal, chip, transaction, chipmaker"
3,61,"year, huawei, company, government, chinese, smartphone, market, network, sale, quarter"
4,11,"project, open_stack, company, kubernete, presentation, foundation, first, event, key, year"
5,35,"apple, company, new, device, pod, phone, year, time, pixel, event"
6,4,"network, carrier, system, mobile, government, security, card, payment, cell, also"
7,8,"broadband, rule, city, net_neutrality, service, wireless, company, internet, pai, deployment"
8,3,"device, network, connectivity, sim, starry, wireless, razer, user, chipset, first"
9,26,"app, game, apple, video, user, mobile, new, gaming, developer, platform"


In [78]:
len(liste)

90